In [9]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException
from time import sleep
from bs4 import BeautifulSoup
import re
import json

def time_wait(driver, num, code):
    try:
        wait = WebDriverWait(driver, num).until(
                  EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

def switch_frame(driver, frame):
    driver.switch_to.default_content()  # frame 초기화
    time.sleep(2)
    driver.switch_to.frame(frame)  # frame 변경

def page_down(driver, num):
    body = driver.find_element(By.TAG_NAME, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)

# 최초 페이지 스크롤 설정
def infinite_loop(driver):
    # 스크롤 전 전체 높이
    last_page_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 윈도우 창을 0에서 스크롤 전 전체 높이로 이동
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2.0)

        # 스크롤 후의 높이를 신규 높이로 설정
        new_page_height = driver.execute_script("return document.body.scrollHeight")

        # 직전 페이지 높이와 신규 페이지 높이 비교
        if new_page_height == last_page_height:
            time.sleep(2.0)

            # 신규 페이지 높이가 이전과 동일하면 while문 break
            if new_page_height == driver.execute_script("return document.body.scrollHeight"):
                break
        else:
            last_page_height = new_page_height


def naverMapCrawling(search):
    
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    webdriver_service = Service("C:/Users/lgpc/Downloads/chromedriver.exe")
    driver = webdriver.Chrome(service=webdriver_service, options=options)

    # PC 네이버 지도
    driver.get(f"https:/map.naver.com/search.naver?query={search}")
    driver.implicitly_wait(10)  # 로딩이 끝날 때까지 10초는 기다림
    
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱
    
    switch_frame(driver, 'searchIframe')
    time.sleep(2)
    page_down(driver, 60)
    
    next_btn = driver.find_elements(By.CSS_SELECTOR, '.mBN2s')

    start = time.time()
    print('[크롤링 시작...]')
    
    try:
        for btn in range(len(next_btn)):
            
            store_list = WebDriverWait(driver, 10).until(
                      EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.UEzoS')))
            print(len(store_list))
            
            for store in store_list:  # 매장 리스트 만큼 반복
                
                try:
                    store.find_element(By.CSS_SELECTOR, '.TYaxT').click()
                    switch_frame(driver, 'entryIframe')
                    time_wait(driver, 3, '.GHAhO')

                    # ----- 매장명 -----
                    store_name = driver.find_element(By.CSS_SELECTOR, '.GHAhO').text
                    
                    # ------ 장소 유형 -----
                    try:
                        category = driver.find_element(By.CSS_SELECTOR, '.lnJFt').text
                        # print(category)
                    except:
                        category = ""
                        # print("음식 카테고리 없음")
                    
                    # ----- 별점 -----
                    try:
                        store_rating = driver.find_element(By.CSS_SELECTOR, '.PXMot.LXIwF').text
                        store_rating = float(re.findall('\d+.\d+', store_rating)[0])
                        # print(store_rating)
                    except:
                        store_rating = ""
                        # print("별점 없음")

                    # ----- 가게 설명글 -----
                    try:
                        descript = driver.find_element(By.CSS_SELECTOR, 'div.XtBbS').text.strip("'\"")
                        descript = descript.replace("\n", " ")
                        # print(descript)
                    except:
                        descript = ""
                        # print("가게 설명글 없음")
                        
                        
                    # 리뷰 탭으로 이동
                    try:
                        # veBoZ 클래스를 가진 모든 요소 찾기
                        tabs = driver.find_elements(By.CSS_SELECTOR, '.veBoZ')

                        # 각 탭 요소를 확인하여 텍스트가 "리뷰"인 탭 클릭
                        for tab in tabs:
                            if tab.text == "리뷰":
                                tab.click()
                                time.sleep(1)
                                break
                                
                        # ----- 대표 키워드 -----
                        try:
                            keyword1 = driver.find_elements(By.CSS_SELECTOR, 'span.t3JSf')[0].text.strip("'\"")
                            keyword2 = driver.find_elements(By.CSS_SELECTOR, 'span.t3JSf')[1].text.strip("'\"")
                            keyword3 = driver.find_elements(By.CSS_SELECTOR, 'span.t3JSf')[2].text.strip("'\"")
                            # print(keyword1)
                            # print(keyword2)
                            # print(keyword3)
                        except:
                            keyword1 = ""
                            keyword2 = ""
                            keyword3 = ""
                            print("키워드 없음")
                        
                        try:
                            toggle_btn = driver.find_elements(By.CSS_SELECTOR, 'span.rvCSr')
                            for toggle in toggle_btn:
                                toggle.click()
                                time.sleep(1)
                        except:
                            print("긴 리뷰 없음")

                        try:
                            reviews = ""
                            for i in range(10):
                                try:
                                    review = driver.find_elements(By.CSS_SELECTOR, 'span.zPfVt')[i].text
                                    if i == 0:
                                        reviews = f'{review}'
                                    else:
                                        reviews = f'{reviews} / {review}'
                                except IndexError:
                                    print("리뷰가 충분하지 않음")
                                    break
                            reviews = reviews.replace("\n", " ")
                            # print(reviews)
                        except Exception as e:
                            reviews = ""
                            print("리뷰 없음")

                    except:
                        print("리뷰탭을 찾지 못함")

                    # 매장 정보 딕셔너리에 추가
                    dict_temp = {
                                  'name': store_name,
                                  'type': category,
                                  'rating' : store_rating,
                                  'descript' : descript,
                                  '1st_keyword' : keyword1,
                                  '2ed_keyword' : keyword2,
                                  '3rd_keyword' : keyword3,
                                  'review' : reviews
                                }

                    store_dict.append(dict_temp)
                    print(f'{store_name} ...완료')

                    # 검색 iframe으로 전환
                    switch_frame(driver, 'searchIframe')
                    time.sleep(1)
                    
                except Exception as e:
                    print(f"Error occurred while processing store data: {e}")

            # 다음 페이지로 이동 
            try:
                next_button = driver.find_elements(By.CSS_SELECTOR, '.mBN2s')
                if next_button[btn+1]:
                    next_button[btn+1].click()
                    time.sleep(1)
            except Exception as e:
                print(f"다음 페이지 없음")
                break

    finally:
        print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
        with open('./store_data.json', 'w', encoding='utf-8') as f:
            json.dump(store_dict, f, indent=4, ensure_ascii=False)
        driver.quit()

In [10]:
# 함수 실행
store_dict = []
search = "신촌 맛집"
naverMapCrawling(search)

[크롤링 시작...]
76
교장맥주 ...완료
고기꾼김춘배 숲길직영점 ...완료
강남불백 신촌점 ...완료
삼산회관 신촌점 ...완료
리정원 신촌점 ...완료
아민 이화 ...완료
머노까머나 신촌점 ...완료
10593 베이글커피하우스 신촌점 ...완료
오마카세 오사이초밥 신촌점 ...완료
치즈웨이브 ...완료
한술식당 신촌점 ...완료
부엉이산장 신촌점 ...완료
정육면체 신촌점 ...완료
백년당 마라탕 신촌본점 ...완료
치히로 신촌점 ...완료
식껍 신촌점 ...완료
히츠지야 신촌점 ...완료
미도인 신촌 ...완료
풍년닭도리탕 신촌직영점 ...완료
돈불1971 ...완료
클로리스 신촌본점 ...완료
명물화로 ...완료
댄싱홍콩 신촌점 ...완료
시누아즈 ...완료
쟁반집8292 신촌점 ...완료
중화복춘 ...완료
쭈꾸미블루스 신촌본점 ...완료
댄싱홍콩 신촌점 ...완료
파이홀 ...완료
칵테일바이스케이프 ...완료
중경마라탕 신촌점 ...완료
신촌고기창고 ...완료
안테이쿠 ...완료
호밀밭 ...완료
뽁식당 신촌점 ...완료
청년화로1987 홍대본점 ...완료
현명식탁 ...완료
유닭스토리 닭한마리 신촌점 ...완료
담산 신촌본점 ...완료
가문의우동 ...완료
카츠업 ...완료
소바연구소 ...완료
청년닭발1987 홍대점 ...완료
로운 샤브샤브 신촌점 ...완료
한술식당 프리미엄 연남점 ...완료
현명식탁 ...완료
청록양식 ...완료
신촌정직한족발 ...완료
몽주방 신촌점 ...완료
온미동 ...완료
카라멘야 ...완료
연하동 ...완료
퐁타이 ...완료
오적회관 신촌점 ...완료
고삼이 신촌점 ...완료
소신이쏘 신촌본점 ...완료
이빠네마 그릴 ...완료
경성양꼬치 신촌직영점 ...완료
미도 매운향솥&마라샹궈 마라탕 ...완료
이석덕생면파스타 ...완료
고르드 ...완료
아마 신촌본점 ...완료
파델라 ...완료
스페샬나잇트 본점 ...완료
형제갈비 ...완료
라구식당 ...완료
신촌황소곱창 신촌본점 ...완료
목구멍 신촌점 ...완료
가

In [12]:
# JSON 파일 열기
import pandas as pd
with open('./store_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df.to_csv('./store_data_1.csv', index=False, encoding='utf-8-sig')
df.head()

,name,type,rating,descript,1st_keyword,2ed_keyword,3rd_keyword,review
0,교장맥주,"맥주,호프",,,음식이 맛있어요,기본 안주가 좋아요,술이 다양해요,친구들끼리 분리된 공간에서 이야기할 수 있어서 좋았어요! 소식좌 친구들 데려오면 정...
1,고기꾼김춘배 숲길직영점,"육류,고기요리",,,음식이 맛있어요,고기 질이 좋아요,친절해요,여기 돼지 A 세트 먹었는데 목살이랑 삼겹살 그리고 등심 덧살 먹었다. 목살은 정말...
2,강남불백 신촌점,"백반,가정식",4.45,가성비 좋은 이화여대 밥집,음식이 맛있어요,가성비가 좋아요,혼밥하기 좋아요,"처음엔 밥통 안의 밥 양이 너무 적다고 생각을 했는데, 적진 않지만 많지도 않음....."
3,삼산회관 신촌점,"육류,고기요리",,,음식이 맛있어요,양이 많아요,고기 질이 좋아요,벌써 4번째 방문이네요!! 신촌에서 김치찌개 제일 맛있다고 단정할 수 있어요ㅠㅠ엄마...
4,리정원 신촌점,"육류,고기요리",,,음식이 맛있어요,고기 질이 좋아요,친절해요,밑반찬이 엄청 알차게 나와서 놀랐어용🥹 직원분들도 친절하시구 매장도 시원하니 쾌적해...


In [21]:
# 함수 실행
store_dict = []
search = "이대 맛집"
naverMapCrawling(search)

[크롤링 시작...]
76
현명식탁B1 ...완료
자연담은화로 신촌점 ...완료
아민 이화 ...완료
리정원 대흥점 ...완료
명물화로 ...완료
호밀밭 ...완료
스페인식당 더크래프트 ...완료
마더린러 베이글 ...완료
하늘초밥 ...완료
옥루몽 신촌본점 ...완료
오르랔베이커리 ...완료
목란 ...완료
아건 이대본점 ...완료
연어초밥 ...완료
비밀베이커리 이대점 ...완료
유소바 이대점 ...완료
비아37 신촌본점 ...완료
아콘스톨 ...완료
연희동 라이라이 본점 ...완료
waffle it up ...완료
조은집 ...완료
명륜진사갈비 서울신촌점 ...완료
연세로갈매기 ...완료
김숙성 대흥본점 ...완료
포닐 ...완료
삭 수제튀김 전문점 이화본점 ...완료
부엉이돈가스 신촌점 ...완료
카우떡볶이 ...완료
내사랑 ...완료
키워드 없음
리뷰가 충분하지 않음
방콕익스프레스 신촌본점 ...완료
고냉지 ...완료
가야가야 ...완료
반타이 쏘이54 ...완료
사장님돈까스 이화여대점 ...완료
은성순대국 ...완료
란주탕슉 ...완료
미스터서왕만두 ...완료
스튜디오웝 신촌점 ...완료
후타바 킷사 ...완료
영미김밥 ...완료
마포분식 ...완료
카페페라 ...완료
까이식당 ...완료
초식곳간 ...완료
블랙번즈 ...완료
증류주유소 ...완료
그릭데이 이대본점 ...완료
진돈부리 ...완료
티앙팡 오후의 홍차 ...완료
크리크 ...완료
서울호떡 아현점 ...완료
공릉닭한마리 ...완료
한끼마끼 ...완료
무사시 ...완료
파파노다이닝 ...완료
댓커피 ...완료
이름없는파스타 이화여대점 ...완료
카페페라유니크 ...완료
스탠바이키친 ...완료
신촌순대국 ...완료
마포깐풍기 ...완료
원즈오운 ...완료
아현닭발 ...완료
청년밥상문간 이화여자대학교점 ...완료
굿바이슈가베어 포케&샐러드 ...완료
리뷰가 충분하지 않음
울 베이커리 카페 ...완료
팔도국밥 육수당 이대역점 ...완료
대전해장국 ...완료
청파비노 ...완료
식탁 하늬솔점 .

In [22]:
# JSON 파일 열기
with open('./store_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df.to_csv('./store_data_2.csv', index=False, encoding='utf-8-sig')
df.head()

,name,type,rating,descript,1st_keyword,2ed_keyword,3rd_keyword,review
0,교장맥주,"맥주,호프",,,음식이 맛있어요,기본 안주가 좋아요,술이 다양해요,친구들끼리 분리된 공간에서 이야기할 수 있어서 좋았어요! 소식좌 친구들 데려오면 정...
1,고기꾼김춘배 숲길직영점,"육류,고기요리",,,음식이 맛있어요,고기 질이 좋아요,친절해요,여기 돼지 A 세트 먹었는데 목살이랑 삼겹살 그리고 등심 덧살 먹었다. 목살은 정말...
2,강남불백 신촌점,"백반,가정식",4.45,가성비 좋은 이화여대 밥집,음식이 맛있어요,가성비가 좋아요,혼밥하기 좋아요,"처음엔 밥통 안의 밥 양이 너무 적다고 생각을 했는데, 적진 않지만 많지도 않음....."
3,삼산회관 신촌점,"육류,고기요리",,,음식이 맛있어요,양이 많아요,고기 질이 좋아요,벌써 4번째 방문이네요!! 신촌에서 김치찌개 제일 맛있다고 단정할 수 있어요ㅠㅠ엄마...
4,리정원 신촌점,"육류,고기요리",,,음식이 맛있어요,고기 질이 좋아요,친절해요,밑반찬이 엄청 알차게 나와서 놀랐어용🥹 직원분들도 친절하시구 매장도 시원하니 쾌적해...


In [33]:
# 두 데이터 하나로 합치기
df1 = pd.read_csv("./신촌_맛집.csv", encoding='utf-8')
df2 = pd.read_csv("./이대_맛집.csv", encoding='utf-8')
print(df1.shape, df2.shape)

total = pd.concat([df1,df2])
print(total.shape)

(312, 8) (312, 8)
(624, 8)


In [47]:
# 중복된 매장의 수 확인
duplicate_cnt = total.duplicated(subset='name', keep=False).sum()
print(duplicate_cnt)

# 중복된 매장 제거
total_unique = total.drop_duplicates(subset='name', keep='first')
print(total_unique.shape)

total_unique.to_csv("./신촌_이대_맛집.csv", index=False, encoding='utf-8-sig')

92
(577, 8)


In [48]:
# 중복된 매장 설명 확인
total['descript'].value_counts()[:20]

농림축산식품부 제공 안심식당                                               70
식약처 인증 위생등급 매우우수 음식점                                          11
보쌈과 김치찌개가 맛있는 신촌 밥집                                            2
신촌 고기집 추천하는 삼겹살 맛집                                             2
숯불갈비 무한리필로 유명한 명륜진사갈비 신촌점                                      2
티라미수 아이스크림이 맛있는 신촌 카페                                          2
신촌 방콕익스프레스 가성비 좋은 태국 음식 맛집                                     2
신촌사이 알흠답게 맛있는 빙수집                                              2
서울 신촌 창천동 조용하고 분위기 좋은 카페                                       2
맵찔이도 맛있었던 부엉이돈까스 볼케이노돈까스                                       2
빕구르망 - 합리적인 가격에 훌륭한 음식을 선사하는 친근한 분위기의 레스토랑 미쉐린 가이드 서울 2024     2
동파육과 멘보샤가 유명한 이연복쉐프 중식당                                        1
신촌 미정국수에서 먹어본 멸치국수                                             1
이대 맛집 현지인이 만드는 인도네팔요리                                          1
종류의 크러핀이 맛있는 베이커리카페                                            1
마포의 맛있는 아부라소바            

In [56]:
# 중복된 리뷰 없음
total_unique['review'].value_counts()[:20].values

# 중복된 매장 설명 존재 > 처리할 필요 없음 
total_unique['descript'].value_counts()[:5]

농림축산식품부 제공 안심식당                                               60
식약처 인증 위생등급 매우우수 음식점                                          11
빕구르망 - 합리적인 가격에 훌륭한 음식을 선사하는 친근한 분위기의 레스토랑 미쉐린 가이드 서울 2024     2
가성비 좋은 이화여대 밥집                                                 1
서울텐동 먹고싶어 신촌 연대맛집                                              1
Name: descript, dtype: int64